## The Problem
**Locality of reference principle** states recently requested data is likely to be requested again. If that data is stored in slow to access disc storage, the retrieval will be slow.

## Cache
A cache is like short-term memory: it has a limited amount of space, but is typically faster than the original data source and contains the most recently accessed items.  
<img src="images/caching.png">

## Cache Strategy
### Read Through
As per this strategy, for a read:
- Check if data exists in the cache
- If it does (cache hit), the cache retreives it from memory and returns response
- If it doesn't (cache miss), the cache retreives it from the database, updates the cache and then returns the response

The cache hit or miss is transparent to the user:
```java
User user = cache.get("Sam");
```
Cache like `Ehcache` and `Hazelcast` support this. Distributed cache like `NCache` also do. Redis doesn't support this directly.

### Cache Aside
Similar to read through with the difference being that the process is not automated. As per this strategy, for a read:
- Check if data exists in the cache
- If it does (cache hit), the the cache retreives it from memory and returns response
- If it doesn't (cache miss), the applicattion retreives it from the database, updates the cache and then returns the response

```java
User user = cache.get("Sam");
if (user == null) {
    user = db.getUser("Sam");     // Manually fetch from DB
    cache.set("Sam", user);       // Manually store in cache
}
```
This strategy provides more flexibility over how cache is populated, but involves more work.

### Write Through
As per this strategy writes are made to cache and the cache synchronously writes to the database. Write is considered complete when write to both cache and database succeeds.

This has the advantage that the cache and database remains in sync. The disadvantage is that writes are slower.

Cache like `Ehcache` and `Hazelcast` support this. Distributed cache like `NCache` also do. Redis doesn't support this directly.

### Write Behind
Write behind is similar to write through the difference being write is made asynchronously. This means that writes are faster at the cost of consistency. Data is eventually consistent.

## Cache Eviction
As cache is filled and memory limit is reached, cache needs to make decision: either reject new writes or make space by removing previously stored data. Cache eviction involves determining which cache entries to retain and which to discard when a cache fills up. Common strategies include:

### LRU (Least Recently Used)
Evicts the least recently accessed cache entries first, based on the assumption that items not recently accessed are less likely to be needed soon. Simple Java implementation:

In [ ]:
public class LRUCache {
    private final Map<Integer, Integer> map;
    private final Node head;
    private final Node tail;
    private final int CAPACITY;
    
    private static class Node {
        Integer key;
        Node next;
        Node prev;
    
        public Node(Integer key) {
            this.key = key;
        }
    }
    
    public LRUCache(int capacity) {
        map = new HashMap<>();
        CAPACITY = capacity;
        head = new Node(null);
        tail = new Node(null);
        head.next = tail;
        tail.prev = head;
    }
    
    public int get(int key) {
        Integer value = map.get(key);
        // Key accessed, need to move it to the end
        if (value != null) {
            delete(key);
            put(key, value);
            return value;
        }
    
        return -1;
    }
    
    public void put(int key, int value) {
        // Overwrite if key exists, moving it to the end
        if (map.containsKey(key)) {
            delete(key);
        // Capacity reached, need to remove the first entry
        } else if (map.size() == CAPACITY) {
            delete(head.next.key);
        }
    
        map.put(key, value);
        addAtEnd(new Node(key));
    }
    
    private void delete(int key) {
        Integer value = map.remove(key);
        if (value != null) {
            Node cur = head.next;
            while (cur != tail) {
                if (cur.key == key) {
                    detach(cur);
                    cur = null;
                    return;
                }
    
                cur = cur.next;
            }
        }
    }
    
    private void detach(Node node) {
        Node previous = node.prev;
        Node forward = node.next;
    
        previous.next = forward;
        forward.prev = previous;
    
        node.prev = null;
        node.next = null;
    }
    
    private void addAtEnd(Node node) {
        Node previous = tail.prev;
    
        tail.prev = node;
        previous.next = node;
    
        node.next = tail;
        node.prev = previous;
    }
}

### LFU (Least Frequently Used)
LFU cache eviction policy evicts the cache entries that are least frequently accessed first, assuming that items accessed infrequently are less likely to be needed in the near future. When there is a tie, LRU can be used for the candidate keys. Simple Java implementation:

In [ ]:
// Memory optimised solution, uses the same linked list structure as previous solution
class LFUCache {
    private final Map<Integer, Integer> map;
    private final Node head;
    private final Node tail;
    private final int CAPACITY;

    private static class Node {
        Integer key;
        Node next;
        Node prev;
        int count; // access counter

        public Node(Integer key) {
            this.key = key;
        }
    }

    public LFUCache(int capacity) {
        map = new HashMap<>();
        CAPACITY = capacity;
        head = new Node(null);
        tail = new Node(null);
        head.next = tail;
        tail.prev = head;
    }

    public int get(int key) {
        Integer value = map.get(key);
        if (value != null) {
            Node temp = find(key);
            temp.count++;

            detach(temp);
            addAtEnd(temp);

            return value;
        }

        return -1;
    }

    public void put(int key, int value) {
        // Overwrite
        if (map.containsKey(key)) {
            Node temp = find(key);
            temp.count++;

            detach(temp);
            addAtEnd(temp);

            map.put(key, value);
            return;
        // Capacity reached
        } else if (map.size() == CAPACITY) {
            Node min = tail.prev;
            Node curr = tail.prev;
            while (curr != head) {
                if (curr.count <= min.count) {
                    min = curr;
                }

                curr = curr.prev;
            }

            detach(min);
            map.remove(min.key);
            min = null;
        }

        map.put(key, value);
        addAtEnd(new Node(key));
    }

    private Node find(int key) {
        Node cur = head.next;
        while (cur != tail) {
            if (cur.key == key) {
                return cur;
            }

            cur = cur.next;
        }

        return null;
    }

    private void detach(Node node) {
        Node previous = node.prev;
        Node forward = node.next;

        previous.next = forward;
        forward.prev = previous;

        node.prev = null;
        node.next = null;
    }

    private void addAtEnd(Node node) {
        Node previous = tail.prev;

        tail.prev = node;
        previous.next = node;

        node.next = tail;
        node.prev = previous;
    }
}

### Time to Live (TTL)
Each cache entry is stamped with a specific “expiration date”. Once that time limit is reached, the entry is evicted, no matter how frequently or recently it was accessed.